# Description


You must read in a file of metropolitan regions and associated sports teams from assets/wikipedia_data.html and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in assets/nfl.csv), MLB (baseball, in assets/mlb.csv), NBA (basketball, in assets/nba.csv or NHL (hockey, in assets/nhl.csv). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: what is the win/loss ratio's correlation with the population of the city it is in?
Average the win/loss ratios for those cities which have multiple teams of a single sport. You should only use data from year 2018 for your analysis -- this is important!

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [ ]:
nhl_df=pd.read_csv("nhl.csv")
nba_df=pd.read_csv("nba.csv")
mlb_df=pd.read_csv("mlb.csv")
nfl_df=pd.read_csv("nfl.csv")

cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [ ]:
nhl_df.head()


,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


In [ ]:
nba_df.head()


,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cleveland Cavaliers* (4),50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA


In [ ]:
mlb_df.head()


,team,W,L,W-L%,GB,year,League
0,Boston Red Sox,108,54,0.667,--,2018,MLB
1,New York Yankees,100,62,0.617,8.0,2018,MLB
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB


In [ ]:
nfl_df.head()


,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,2018
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots*,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018


In [ ]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [ ]:
nhl_df = nhl_df[nhl_df['year'] == 2018]
nhl_df

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL
6,Montreal Canadiens,82,29,40,13,71,.433,209,264,-0.68,0.00,.378,27,2018,NHL
7,Ottawa Senators,82,28,43,11,67,.409,221,291,-0.85,0.00,.372,26,2018,NHL
8,Buffalo Sabres,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL
9,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,Metropolitan Division,2018,NHL


In [ ]:
nfl_df

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,2018
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots*,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,NFC West,NFC West,NFL,NFC West,NFC West,NFC West,NFC West,NFC West,NFC West,NFC West,NFC West,NFC West,NFC West,NFC West,2014
196,7.1,4,NFL,8.8,2.4,254,140,394,9.5,0.8,0,12,.750,Seattle Seahawks*,2014
197,4.4,5,NFL,0.7,-2.4,299,11,310,2.0,1.3,0,11,.688,Arizona Cardinals+,2014
198,2.1,8,NFL,-2.1,-3.0,340,-34,306,-1.0,1.2,0,8,.500,San Francisco 49ers,2014


In [ ]:
# Limpieza de datos

import warnings
warnings.filterwarnings("ignore")


cities.rename(columns = {'Population (2016 est.)[8]': 'Population'}, inplace = True)


nhl_df = nhl_df[nhl_df['year'] == 2018]
nhl_df.drop([0,9,18,26], axis = 0, inplace = True)
nhl_df['team'] = nhl_df['team'].str.replace(r'\*$', '',) 
ratio_nhl = nhl_df.groupby('team').apply(lambda x: float(x['W'])/(float(x['W'])+float(x['L'])))

nba_df = nba_df[nba_df['year'] == 2018]
nba_df['team'] = nba_df['team'].str.replace(r'(\*.*)|(\s\(.*)', '',)
ratio_nba = nba_df.groupby('team').apply(lambda x: float(x['W'])/(float(x['W'])+float(x['L'])))

mlb_df = mlb_df[mlb_df['year'] == 2018]
mlb_df['team'] = mlb_df['team'].str.replace(r'(\*.*)|(\s\(.*)', '',)
ratio_mlb = mlb_df.groupby('team').apply(lambda x: float(x['W'])/(float(x['W'])+float(x['L'])))

nfl_df = nfl_df[nfl_df['year'] == 2018]
nfl_df.drop([0,5,10,15,20,25,30,35], axis = 0, inplace = True)
nfl_df['team'] = nfl_df['team'].str.replace(r'(\*.*)|(\s\(.*)|(\+$)', '',)
ratio_nfl = nfl_df.groupby('team').apply(lambda x: float(x['W'])/(float(x['W'])+float(x['L'])))


cities['NHL']  = cities['NHL'].str.replace(r'\[.*','')
cities['NBA']  = cities['NBA'].str.replace(r'\[.*','')
cities['MLB']  = cities['MLB'].str.replace(r'\[.*','')
cities['NFL']  = cities['NFL'].str.replace(r'\[.*','')



In [ ]:
cities_draft = cities.copy()
    
ny1 = pd.Series({'Metropolitan area': 'New York City', 'Population': 20153634, 'NFL': 'Giants', 'MLB': 'Yankees', 'NBA': 'Knicks', 'NHL': 'Rangers'})
ny2 = pd.Series({'Metropolitan area': 'New York City', 'Population': 20153634, 'NFL': 'Jets', 'MLB': 'Mets', 'NBA': 'Nets', 'NHL': 'Islanders'})
ny3 = pd.Series({'Metropolitan area': 'New York City', 'Population': 20153634, 'NFL': '—', 'MLB': '—', 'NBA': '—', 'NHL': 'Devils'})
la1 = pd.Series({'Metropolitan area': 'Los Angeles', 'Population': 13310447, 'NFL': 'Rams', 'MLB': 'Dodgers', 'NBA': 'Lakers', 'NHL': 'Kings'})
la2 = pd.Series({'Metropolitan area': 'Los Angeles', 'Population': 13310447, 'NFL': 'Chargers', 'MLB': 'Angels', 'NBA': 'Clippers', 'NHL': 'Ducks'})
sf1 = pd.Series({'Metropolitan area': 'San Francisco Bay Area', 'Population': 6657982, 'NFL': '49ers', 'MLB': 'Giants', 'NBA': 'Warriors', 'NHL': 'Sharks'})
sf2 = pd.Series({'Metropolitan area': 'San Francisco Bay Area', 'Population': 6657982, 'NFL': '—', 'MLB': 'Athletics', 'NBA': '—', 'NHL': '—'})
ch1 = pd.Series({'Metropolitan area': 'Chicago', 'Population': 9512999, 'NFL': 'Bears', 'MLB': 'Cubs', 'NBA': 'Bulls', 'NHL': 'Blackhawks'})
ch2 = pd.Series({'Metropolitan area': 'Chicago', 'Population': 9512999, 'NFL': '—', 'MLB': 'White Sox', 'NBA': '—', 'NHL': '—'})
lv1 = pd.Series({'Metropolitan area': 'Las Vegas', 'Population': 2155664, 'NFL': 'Raiders', 'MLB': '—', 'NBA': '—', 'NHL': 'Golden Knights'})

teams2 = pd.DataFrame([ny1, ny2, ny3, la1, la2, sf1, sf2, ch1, ch2])



In [ ]:
teams2

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,New York City,20153634,Giants,Yankees,Knicks,Rangers
1,New York City,20153634,Jets,Mets,Nets,Islanders
2,New York City,20153634,—,—,—,Devils
3,Los Angeles,13310447,Rams,Dodgers,Lakers,Kings
4,Los Angeles,13310447,Chargers,Angels,Clippers,Ducks
5,San Francisco Bay Area,6657982,49ers,Giants,Warriors,Sharks
6,San Francisco Bay Area,6657982,—,Athletics,—,—
7,Chicago,9512999,Bears,Cubs,Bulls,Blackhawks
8,Chicago,9512999,—,White Sox,—,—


In [ ]:
# Limpiando datos data wikipedia

wiki_clean = pd.concat([cities_draft,teams2])
wiki_clean = wiki_clean.set_index('Metropolitan area').reset_index()
wiki_clean.drop([0,1,2,3], inplace = True)
wiki_clean.set_index('Metropolitan area').reset_index()


#wiki_clean

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
1,"Washington, D.C.",6131977,Redskins,Nationals,Wizards,Capitals
2,Philadelphia,6070500,Eagles,Phillies,76ers,Flyers
3,Boston,4794447,Patriots,Red Sox,Celtics,Bruins
4,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild
5,Denver,2853077,Broncos,Rockies,Nuggets,Avalanche
6,Miami–Fort Lauderdale,6066387,Dolphins,Marlins,Heat,Panthers
7,Phoenix,4661537,Cardinals,Diamondbacks,Suns,Coyotes
8,Detroit,4297617,Lions,Tigers,Pistons,Red Wings
9,Toronto,5928040,—,Blue Jays,Raptors,Maple Leafs


In [ ]:
ratio_nhl

team
Anaheim Ducks            0.637681
Arizona Coyotes          0.414286
Boston Bruins            0.714286
Buffalo Sabres           0.357143
Calgary Flames           0.513889
Carolina Hurricanes      0.507042
Chicago Blackhawks       0.458333
Colorado Avalanche       0.589041
Columbus Blue Jackets    0.600000
Dallas Stars             0.567568
Detroit Red Wings        0.434783
Edmonton Oilers          0.473684
Florida Panthers         0.594595
Los Angeles Kings        0.608108
Minnesota Wild           0.633803
Montreal Canadiens       0.420290
Nashville Predators      0.746479
New Jersey Devils        0.602740
New York Islanders       0.486111
New York Rangers         0.465753
Ottawa Senators          0.394366
Philadelphia Flyers      0.617647
Pittsburgh Penguins      0.618421
San Jose Sharks          0.625000
St. Louis Blues          0.578947
Tampa Bay Lightning      0.701299
Toronto Maple Leafs      0.653333
Vancouver Canucks        0.436620
Vegas Golden Knights     0.680000
Washingto

In [ ]:
ratio_nhl = pd.DataFrame(ratio_nhl)
ratio_nhl = ratio_nhl.reset_index()
ratio_nba = pd.DataFrame(ratio_nba)
ratio_nba = ratio_nba.reset_index()
ratio_mlb = pd.DataFrame(ratio_mlb)
ratio_mlb = ratio_mlb.reset_index()
ratio_nfl = pd.DataFrame(ratio_nfl)
ratio_nfl = ratio_nfl.reset_index()

In [ ]:
# Limpiando ratios

ratio_nhl.replace({'New York Rangers': 'Rangers',
                   'New York Islanders': 'Islanders',
                   'New Jersey Devils': 'Devils',
                   'Los Angeles Kings': 'Kings',
                   'Anaheim Ducks': 'Ducks',
                   'Chicago Blackhawks': 'Blackhawks',
                   'San Jose Sharks': 'Sharks',
                   'Dallas Stars': 'Stars',
                   'Washington Capitals': 'Capitals',
                   'Philadelphia Flyers': 'Flyers',
                   'Florida Panthers': 'Panthers',
                   'Boston Bruins': 'Bruins',
                   'Minnesota Wild': 'Wild',
                   'Colorado Avalanche': 'Avalanche',
                   'Arizona Coyotes': 'Coyotes',
                   'Detroit Red Wings': 'Red Wings',
                   'Toronto Maple Leafs': 'Maple Leafs',
                   'Tampa Bay Lightning': 'Lightning',
                   'Pittsburgh Penguins': 'Penguins',
                   'Nashville Predators': 'Predators',
                   'St. Louis Blues': 'Blues',
                   'Vegas Golden Knights': 'Golden Knights',
                   'Buffalo Sabres': 'Sabres',
                   'Montreal Canadiens': 'Canadiens',
                   'Vancouver Canucks': 'Canucks',
                   'Columbus Blue Jackets': 'Blue Jackets',
                   'Calgary Flames': 'Flames',
                   'Ottawa Senators': 'Senators',
                   'Edmonton Oilers': 'Oilers',
                   'Winnipeg Jets': 'Jets',
                   'Carolina Hurricanes': 'Hurricanes'}, inplace = True)
ratio_nba.replace({'New York Knicks': 'Knicks',
                       'Brooklyn Nets': 'Nets',
                       'Los Angeles Lakers': 'Lakers',
                       'Los Angeles Clippers': 'Clippers',
                       'Chicago Bulls': 'Bulls',
                       'Golden State Warriors': 'Warriors',
                       'Dallas Mavericks': 'Mavericks',
                       'Washington Wizards': 'Wizards',
                       'Philadelphia 76ers': '76ers',
                       'Miami Heat': 'Heat',
                       'Boston Celtics': 'Celtics',
                       'Minnesota Timberwolves': 'Timberwolves',
                       'Denver Nuggets': 'Nuggets',
                       'Phoenix Suns': 'Suns',
                       'Detroit Pistons': 'Pistons',
                       'Toronto Raptors': 'Raptors',
                       'Houston Rockets': 'Rockets',
                       'Atlanta Hawks': 'Hawks',
                       'Cleveland Cavaliers': 'Cavaliers',
                       'Charlotte Hornets': 'Hornets',
                       'Indiana Pacers': 'Pacers',
                       'Milwaukee Bucks': 'Bucks',
                       'New Orleans Pelicans': 'Pelicans',
                       'Orlando Magic': 'Magic',
                       'Portland Trail Blazers': 'Trail Blazers',
                       'Utah Jazz': 'Jazz',
                       'San Antonio Spurs': 'Spurs',
                       'Sacramento Kings': 'Kings',
                       'Oklahoma City Thunder': 'Thunder',
                       'Memphis Grizzlies': 'Grizzlies',
                       }, inplace = True)
ratio_mlb.replace({'New York Yankees': 'Yankees',
                       'New York Mets': 'Mets',
                       'Los Angeles Dodgers': 'Dodgers',
                       'Los Angeles Angels': 'Angels',
                       'Chicago Cubs': 'Cubs',
                       'Chicago White Sox': 'White Sox',
                       'San Francisco Giants': 'Giants',
                       'Oakland Athletics': 'Athletics',
                       'Texas Rangers': 'Rangers',
                       'Washington Nationals': 'Nationals',
                       'Philadelphia Phillies': 'Phillies',
                       'Miami Marlins': 'Marlins',
                       'Boston Red Sox': 'Red Sox',
                       'Minnesota Twins': 'Twins',
                       'Colorado Rockies': 'Rockies',
                       'Arizona Diamondbacks': 'Diamondbacks',
                       'Detroit Tigers': 'Tigers',
                       'Toronto Blue Jays': 'Blue Jays',
                       'Houston Astros': 'Astros',
                       'Atlanta Braves': 'Braves',
                       'Seattle Mariners': 'Mariners',
                       'Tampa Bay Rays': 'Rays',
                       'Pittsburgh Pirates': 'Pirates',
                       'Cleveland Indians': 'Indians',
                       'Cincinnati Reds': 'Reds',
                       'Kansas City Royals': 'Royals',
                       'St. Louis Cardinals': 'Cardinals',
                       'Baltimore Orioles': 'Orioles',
                       'Milwaukee Brewers': 'Brewers',
                       'San Diego Padres': 'Padres',
                       }, inplace = True)
ratio_nfl['team'].replace({'New York Giants': 'Giants',
                   'New York Jets': 'Jets',
                   'Los Angeles Rams': 'Rams',
                   'Los Angeles Chargers': 'Chargers',
                   'Chicago Bears': 'Bears',
                   'Dallas Cowboys': 'Cowboys',
                   'Washington Redskins': 'Redskins',
                   'Philadelphia Eagles': 'Eagles',
                   'Miami Dolphins': 'Dolphins',
                   'New England Patriots': 'Patriots',
                   'Minnesota Vikings': 'Vikings',
                   'Denver Broncos': 'Broncos',
                   'Arizona Cardinals': 'Cardinals',
                   'Detroit Lions': 'Lions',
                   'Houston Texans': 'Texans',
                   'Atlanta Falcons': 'Falcons',
                   'Seattle Seahawks': 'Seahawks',
                   'Tampa Bay Buccaneers': 'Buccaneers',
                   'Pittsburgh Steelers': 'Steelers',
                   'Cleveland Browns': 'Browns',
                   'Cincinnati Bengals': 'Bengals',
                   'Kansas City Chiefs': 'Chiefs',
                   'Tennessee Titans': 'Titans',
                   'Carolina Panthers': 'Panthers',
                   'Baltimore Ravens': 'Ravens',
                   'Oakland Raiders': 'Raiders',
                   'Indianapolis Colts': 'Colts',
                   'New Orleans Saints': 'Saints',
                   'Buffalo Bills': 'Bills',
                   'Jacksonville Jaguars': 'Jaguars',
                   'Green Bay Packers': 'Packers',
                   'San Francisco 49ers': '49ers'}, inplace = True)

ratio_nhl.rename(columns = {0: 'ratio_nhl', 'team': 'NHL'}, inplace = True)
ratio_nba.rename(columns = {0: 'ratio_nba', 'team': 'NBA'}, inplace = True)
ratio_mlb.rename(columns = {0: 'ratio_mlb', 'team': 'MLB'}, inplace = True)
ratio_nfl.rename(columns = {0: 'ratio_nfl', 'team': 'NFL'}, inplace = True)




In [ ]:
ratio_nba

,NBA,ratio_nba
0,Hawks,0.292683
1,Celtics,0.670732
2,Nets,0.341463
3,Hornets,0.439024
4,Bulls,0.329268
5,Cavaliers,0.609756
6,Mavericks,0.292683
7,Nuggets,0.560976
8,Pistons,0.475610
9,Warriors,0.707317


In [ ]:
# Uniendo datos de ratio y poblacion

rateandpop_nhl = pd.merge(wiki_clean,ratio_nhl, how = 'outer', on = 'NHL')
rateandpop_nba = pd.merge(wiki_clean,ratio_nba, how = 'outer', on = 'NBA')
rateandpop_mlb = pd.merge(wiki_clean,ratio_mlb, how = 'outer', on = 'MLB')
rateandpop_nfl = pd.merge(wiki_clean,ratio_nfl, how = 'outer', on = 'NFL')

In [ ]:
# Organizando para posterior correlación

ans_nhl = rateandpop_nhl[['Metropolitan area', 'ratio_nhl', 'Population']]
ans_nhl.dropna(inplace = True)
ans_nhl['Population'] = ans_nhl['Population'].apply(lambda x: float(x))
ans_nhl = ans_nhl.groupby('Metropolitan area').apply(np.mean)

ans_nba = rateandpop_nba[['Metropolitan area', 'ratio_nba', 'Population']]
ans_nba.dropna(inplace = True)
ans_nba['Population'] = ans_nba['Population'].apply(lambda x: float(x))
ans_nba = ans_nba.groupby('Metropolitan area').apply(np.mean)

ans_mlb = rateandpop_mlb[['Metropolitan area', 'ratio_mlb', 'Population']]
ans_mlb.dropna(inplace = True)
ans_mlb['Population'] = ans_mlb['Population'].apply(lambda x: float(x))
ans_mlb = ans_mlb.groupby('Metropolitan area').apply(np.mean)

ans_nfl = rateandpop_nfl[['Metropolitan area', 'ratio_nfl', 'Population']]
ans_nfl.dropna(inplace = True)
ans_nfl['Population'] = ans_nfl['Population'].apply(lambda x: float(x))
ans_nfl = ans_nfl.groupby('Metropolitan area').apply(np.mean)




In [ ]:
#Hallando correlaciones

population_by_region_nhl = list(ans_nhl['Population'])
win_loss_by_region_nhl = list(ans_nhl['ratio_nhl'])
cor_nhl = stats.pearsonr(population_by_region_nhl, win_loss_by_region_nhl)[0]

population_by_region_nba = list(ans_nba['Population'])
win_loss_by_region_nba = list(ans_nba['ratio_nba'])
cor_nba = stats.pearsonr(population_by_region_nba, win_loss_by_region_nba)[0]

population_by_region_mlb = list(ans_mlb['Population'])
win_loss_by_region_mlb = list(ans_mlb['ratio_mlb'])
cor_mlb = stats.pearsonr(population_by_region_mlb, win_loss_by_region_mlb)[0]

population_by_region_nfl = list(ans_nfl['Population'])
win_loss_by_region_nfl = list(ans_nfl['ratio_nfl'])
cor_nfl = stats.pearsonr(population_by_region_nfl, win_loss_by_region_nfl)[0]

In [ ]:
# win/loss ratio's correlation with the population of the city

final = {'NHL: ': cor_nhl,
         'NBA: ': cor_nba,
         'MLB: ': cor_mlb,
         'NHL: ': cor_nhl,
         'NFL: ': cor_nfl}

final

{'NHL: ': 0.012486162921209923,
 'NBA: ': -0.17657160252844614,
 'MLB: ': 0.15027698302669307,
 'NFL: ': 0.004922112149349409}